In [1]:
import autograd.numpy as np
from autograd import grad

In [2]:
class NANDNeuron():
    def __init__(self, n):
        self.n = n
        self.epsilons = np.full((n), -1.0) #np.random.uniform(-1.0, 1.001, n)#
        self.effector = -1
        
    def __repr__(self):
        return str(self.epsilons)
        
    def __beforePresent__(self):
        self.effector = -1
        self.delta = 0
    
    def getDelta(self):
        return self.delta
    
    def setDelta(self, delta):
        self.delta = delta
    
    def present(self, inputs):
        self.__beforePresent__()
        mus = np.maximum(self.epsilons, inputs)
        self.mus = mus
        self.inputs = inputs
        out = np.min(mus)
        
        # Determin which had effect
        for i in range(0, len(mus)):
            if out == mus[i]:
                self.effector = i
        
        self.output = -out
        return self.output

    def getInput(self):
        return self.inputs
    
    def getMus(self):
        return self.mus
    
    def getOutput(self):
        return self.output
    
    def updateWeights(self, grad):
        self.epsilons = self.epsilons - grad
        for i in range(0, len(self.epsilons)):
            if self.epsilons[i] > 1:
                self.epsilons[i] = 1
            elif self.epsilons[i] < -1:
                self.epsilons[i] = -1
    
    def getWeights(self):
        return self.epsilons
    
    def getEffector(self):
        return self.effector

class NANDLayer():
    def __init__(self, inputs, nodes):
        self.layer = []
        
        for i in range(0, nodes):
            self.layer.append(NANDNeuron(inputs))
            
    def __repr__(self):
        s = ""
        for l in self.layer:
            s += (str(l) + " ,")
            
        return s

    def getLayer(self):
        return self.layer
    
    def setFolowingLayer(self, l):
        self.folowingLayer = l
        
    def present(self, inputs):
        out = []
        for n in self.layer:
            out.append(n.present(inputs))
        
        return out
    
    def backprop(self, prediction, target, output=False):
        for n in range(0, len(self.layer)):
            grad = None
            if output:
                grad = gradientOutputLayer(self.layer[n], target, prediction)
            else:
                grad = gradientHiddenLayer(self.layer[n], n, self.folowingLayer)
                
            self.layer[n].updateWeights(grad * 0.05)
    
class NANDNetwork():
    def __init__(self, nIns, lParams, nOuts):
        self.layers = []
        
        lParams.append(nOuts)
        inputs = nIns
        for l in lParams:
            self.layers.append(NANDLayer(inputs, l))
            inputs = l
                    
        for i in range(1, len(self.layers)):
            self.layers[i-1].setFolowingLayer(self.layers[i])
        
    def __repr__(self):
        s = ""
        for l in range(0, len(self.layers)):
            s += ("Layer " + str(l+1) + " -> " + str(self.layers[l]) + "\n")
            
        return s
        
    def fowardprop(self, inputs):
        for l in self.layers:
            inputs = l.present(inputs)
            
        return inputs[0]
            
    def backprop(self, prediction, target):
        for i in range(len(self.layers)-1, -1, -1):
            layer = self.layers[i]
            layer.backprop(prediction, target, i==(len(self.layers) - 1))
    
    
def MSE(network, data, targets):
    expected = np.array(list(map(lambda x: network.fowardprop(x), data)))
#     print(expected)
    return (1.0/2.0) * np.sum(np.power(np.subtract(expected, targets), 2))

def gradientHiddenLayer(neuron, neuronNumber, folowingLayer):
    numWeights = len(neuron.getWeights())
    grad = np.zeros(numWeights)
    
    # Compute the delta of current node
    dE_dy = 0
    for i in range(0, len(folowingLayer.getLayer())):
        fn = folowingLayer.getLayer()[i]
        dE_dyi = fn.getDelta()
        
        if fn.getEffector() == neuronNumber:
            if not fn.getInput()[neuronNumber] == neuron.getOutput():
                print("ERRRRRORRORORO")
                
            dxi_dyi = (fn.getWeights()[neuronNumber] - fn.getInput()[neuronNumber])
            dE_dy += (dE_dyi * dxi_dyi)
        
        
    neuron.setDelta(dE_dy)
    
    for i in range(0, numWeights):
        dx_dw = 0
        if neuron.getEffector() == i:
            dx_dw = -(neuron.getWeights()[i] - neuron.getInput()[i])
        
        grad[i] = dE_dy * dx_dw
    
#     for i in range(0, len(folowingLayer.getLayer())):
#         fowardNeuron = folowingLayer.getLayer()[i]
        
#         dE_dyi = fowardNeuron.getDelta()
#         dxi_dmui = np.min(np.delete(fowardNeuron.getMus(), neuronNumber)) - fowardNeuron.getMus()[neuronNumber]
#         dmui_dyi = neuron.getOutput() - fowardNeuron.getWeights()[neuronNumber]
#         dxi_dyi = dxi_dmui * dmui_dyi
#         dE_dy += dE_dyi * dxi_dyi
    
#     for i in range(0, numWeights):
#         dx_dmu = np.min(np.delete(neuron.getMus(), i)) - neuron.getMus()[i]
#         dmu_dw = neuron.getWeights()[i] - neuron.getInput()[i]
#         dx_dw = dx_dmu * dmu_dw
        
#         grad[i] = -dE_dy * dx_dw
# #         grad[i] = dE_dy * deltaW(i, neuron) * (neuron.getWeights()[i] - neuron.getInput()[i])
        
    neuron.setDelta(dE_dy)
    
    return grad
    
    
def gradientOutputLayer(neuron, target, prediction):
    numWeights = len(neuron.getWeights())
    grad = np.zeros(numWeights)
    
    dE_dy = -(target - prediction)
#     for i in range(0, numWeights):
#         dx_dmu = np.min(np.delete(neuron.getMus(), i)) - neuron.getMus()[i]
#         dmu_dw = neuron.getWeights()[i] - neuron.getInput()[i]
#         dx_dw = dx_dmu * dmu_dw
        
#         grad[i] = dE_dy * dx_dw
#         grad[i] = dE_dy * deltaW(i, neuron) * (neuron.getWeights()[i] - neuron.getInput()[i])
        
    for i in range(0, numWeights):
        dx_dw = 0
        if neuron.getEffector() == i:
            dx_dw = -(neuron.getWeights()[i] - neuron.getInput()[i])
        
        grad[i] = dE_dy * dx_dw

    neuron.setDelta(dE_dy)
#     print(grad)
    return grad

# Compute W_{a,b}
# def deltaW(i, neuron):
#     muBs = np.delete(neuron.getMus(), i)
#     print()
#     print(neuron.getMus())
#     print(muBs)
#     print(neuron.getMus()[i])
#     t = (np.min(muBs) - neuron.getMus()[i]) * (neuron.getWeights()[i] - neuron.getInput()[i])
#     print(t)
#     return t

In [3]:
def trainNANDNetwork(data, targets, inputNodes, hLayers, outNodes):
    network = NANDNetwork(inputNodes, hLayers, outNodes)
    print(network)
    print("Initial Loss: ", MSE(network, data, targets))
    
    for i in range(1, 100000):
        
        if (MSE(network, data, targets) < 0.0000000000001):
            break
#         if i%1000 == 0:
#             print("Iteration -> " + str(i))
            
        for j in range(0, len(data)):
            prediction = network.fowardprop(data[j])
            network.backprop(prediction, targets[j])
            
    print("Trained Loss: ", MSE(network, data, targets))
    return network
            
            

In [4]:
# NAND Gate Data
# data = np.array([[-1.0,-1.0],[1.0,-1.0],[-1.0,1.0],[1.0,1.0]])
# targets = np.array([1.0,1.0,1.0,-1.0])



# NOT Gate Data
# data = np.array([[-1.0, 1.0], [1.0, 1.0]])
# targets = np.array([1.0, -1.0])
# trainNANDNetwork(data, targets, 2, [], 1)


# AND Gate Data
# data = np.array([[-1.0, -1.0], [-1.0, 1.0], [1.0, -1.0], [1.0, 1.0]])
# targets = np.array([-1.0, -1.0, -1.0, 1.0])

# OR Gate Data 
data = np.array([[-1.0, -1.0], [-1.0, 1.0], [1.0, -1.0], [1.0, 1.0]])
targets = np.array([-1.0, 1.0, 1.0, 1.0])

trainNANDNetwork(data, targets, 2, [2], 1)


Layer 1 -> [-1. -1.] ,[-1. -1.] ,
Layer 2 -> [-1. -1.] ,

Initial Loss:  4.0
Trained Loss:  4.0


Layer 1 -> [-1. -1.] ,[-1. -1.] ,
Layer 2 -> [-1.  1.] ,